In [22]:
import pandas as pd
import os
import glob


def remove_parenthesis_from_text(text):
    if "(" in text:
        text_to_remove = text[text.find("(") : text.find(")") + 1]
        text = text.replace(text_to_remove, "")
        text = " ".join([w.strip() for w in text.split()])
    return text


root_dir = "tm2/"
path = os.path.join(root_dir, "raw-data/main/*.zip")
files = glob.glob(path)

keywords = []
for file in files:
    df = pd.read_csv(file, compression="zip")

    author_keywords = (
        df["Author Keywords"]
        .dropna()
        .str.split(";")
        .explode()
        .map(remove_parenthesis_from_text)
        .str.split(" ")
        .explode()
        .str.strip()
        .str.upper()
        .drop_duplicates()
        .to_list()
    )

    index_keywords = (
        df["Index Keywords"]
        .dropna()
        .str.upper()
        .str.split(";")
        .explode()
        .map(remove_parenthesis_from_text)
        .str.split(" ")
        .explode()
        .str.strip()
        .str.upper()
        .drop_duplicates()
        .to_list()
    )

    keywords += list(set(author_keywords + index_keywords))
    keywords = list(set(keywords))


hypened_keywords = [word for word in keywords if "-" in word]
hypened_keywords = [word for word in hypened_keywords if word.replace("-", "") in keywords]
hypened_keywords = [word for word in hypened_keywords if word[0] != "-"]
hypened_keywords = [word for word in hypened_keywords if word[-1] != "-"]
hypened_keywords = sorted(hypened_keywords)

with open("hypened_keywords.txt", "w") as f:
    for word in hypened_keywords:
        f.write(word + "\n")